## Step 1: Install necessary libraries.

In [ ]:
!pip install -q Flask ibm-watson ibm-db

## Step 2: Create valid Port connect to website in Google Colab.

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)"))

# Step 3: Connect to Db2 Database from IBM and query table contains students information.

In [ ]:
from flask import Flask, render_template, request, jsonify, Response
import ibm_db
import pandas as pd
from datetime import date
import os

# Set your IBM Db2 credentials in the environment variables or manually assign them in the code below.
dsn_hostname = os.getenv('DB_HOST', 'YOUR DATABASE HOST') 
dsn_uid = os.getenv('DB_UID', 'YOUR USERNAME')  
dsn_pwd = os.getenv('DB_PWD', 'YOUR PASSWORD')  
dsn_port = os.getenv('DB_PORT', 'YOUR PORT')  
dsn_database = "bludb"  # Change if necessary
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_protocol = "TCPIP"
dsn_security = "SSL"

# Create the dsn connection string
conn_str = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd, dsn_security)

# Connect the template html
app = Flask(__name__, template_folder = "template")

@app.route('/')
def index():
    return render_template("index.html")

@app.route('/submit', methods=['POST'])
def submit():
    try:
      name = request.form['name']
      birthday = request.form['birthday']
      user_id = request.form['id']
      major = request.form["major"]

      # Connect to the database
      conn = ibm_db.connect(conn_str, '', '')

      # Insert data into the database
      sql = "INSERT INTO STUDENTS VALUES (?, ?, ?, ?)"
      stmt = ibm_db.prepare(conn, sql)
      ibm_db.bind_param(stmt, 1, name)
      ibm_db.bind_param(stmt, 2, birthday)
      ibm_db.bind_param(stmt, 3, user_id)
      ibm_db.bind_param(stmt, 4, major)
      ibm_db.execute(stmt)

      print(birthday)
      # Close the connection
      ibm_db.close(conn)

    except Exception as err:
      print(f"Unexpected {err=}, {type(err)=}")

    return index()

@app.route('/delete', methods=['POST'])
def delete():
  try:
      user_id = request.form['id']

      # Connect to the database
      conn = ibm_db.connect(conn_str, '', '')

      # Delete data from the database
      sql = "DELETE FROM TABLENAME WHERE \"YOUR COLUMN QUERY\" = ?"
      stmt = ibm_db.prepare(conn, sql)
      ibm_db.bind_param(stmt, 1, user_id)
      ibm_db.execute(stmt)

      # Close the connection
      ibm_db.close(conn)

  except Exception as err:
      print(f"Unexpected {err=}, {type(err)=}")
  return index()

@app.route('/update')
def update():
    conn = ibm_db.connect(conn_str, "", "")

    # Query to get the first 5 rows from the table
    command = "SELECT * FROM TABLENAME ORDER BY \"YOUR COLUMN QUERY\" DESC"
    stmt = ibm_db.exec_immediate(conn, command)

    # Fetch the result set
    result = ibm_db.fetch_both(stmt)
    rows = []

    # Loop through the result and fetch the rows
    while result:
        rows.append({k:v for (k,v) in result.items() if not isinstance(k, int)})
        result = ibm_db.fetch_both(stmt)

    # Convert result to a pandas DataFrame for better readability
    return pd.DataFrame(rows).to_html()

# Step 4: Run to create a port and click on the link in the Step 2 to connect the website sever.

In [ ]:
if __name__ == '__main__':
    app.run(port = 8000)
